In [1]:
import numpy as np
from deap import algorithms,base,creator,tools,gp
import random
import operator
import math
import gym
from gym import wrappers
import time

In [2]:
ENV = gym.make("CartPole-v0")
MAX_STEPS=200
NGEN=30

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
class Myclass:
    count=0
    FT_avg=[]
    FT_max=[]
    gen=[]
    for i in range(1,NGEN+1):
        gen.append(i)

In [4]:
def safeDiv(left,right):
    if(right==0):
        return 0
    try:
        return left/right
    except ZeroDivisionError:
        return 0

pset = gp.PrimitiveSet("MAIN", 2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(safeDiv,  2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("rand101", lambda:random.randint(-1,1))
"""
pset.renameArguments(ARG0='x')
pset.renameArguments(ARG1='v')
pset.renameArguments(ARG2='theta')
pset.renameArguments(ARG3='av')
"""
pset.renameArguments(ARG0='theta')
pset.renameArguments(ARG1='av')
'''
pset.addEphemeralConstant("theta2", lambda:ARG0*2)
pset.addEphemeralConstant("theta3", lambda:ARG0*3)
pset.addEphemeralConstant("theta4", lambda:ARG0*4)
'''
creator.create("FitnessMax",base.Fitness,weights=(1.0,))
creator.create("Individual",gp.PrimitiveTree,fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population",tools.initRepeat,list,toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)


In [5]:
def EV(individual):
        func = toolbox.compile(expr=individual)
        observation = ENV.reset()
        episode_reward=0
        for step in range(MAX_STEPS):
            action = get_action(observation,individual)

            observation_next,reward,done,info_not = ENV.step(action)
            if done:
                ENV.render()
                break

            observation = observation_next
            episode_reward += reward
            if Myclass.count>=2700:
                env = wrappers.Monitor(env,'./movie/cartpole-experiment-1')
            ENV.render()
        Myclass.count+=1
        return episode_reward,

In [7]:
toolbox.register("evaluate", EV)
toolbox.register("mate", gp.cxOnePoint)
#toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb = 0.1)
toolbox.register("select",tools.selTournament,tournsize=3)
toolbox.register("expr_mut", gp.genFull, min_= 0, max_ = 2)
toolbox.register("mutate",gp.mutUniform,expr=toolbox.expr_mut,pset=pset)

In [8]:
def get_action(observation,individual):
    action = decide_action(observation,individual) ##
    return action
    #observation[カートの位置，カートの速度，棒の角度，棒の角速度]
def decide_action(observation,individual):
    func = toolbox.compile(expr=individual)
    if(func(observation[2],observation[3]) >= 10):
        return 1
    else :
        return 0

In [9]:
def main():
        random.seed(1)
        pop = toolbox.population(n=100)
        hof = tools.HallOfFame(1)

        stats_fit= tools.Statistics(lambda ind: ind.fitness.values)
        stats_size = tools.Statistics(len)

        mstats = tools.MultiStatistics(fitness=stats_fit,size=stats_size)
        mstats.register("avg", np.mean)
        mstats.register("std", np.std)
        mstats.register("min", np.min)
        mstats.register("max", np.max)
        CXPB=0.5
        MUTPB=0.1
        pop,log = algorithms.eaSimple(pop, toolbox, CXPB, MUTPB, NGEN, stats = mstats, halloffame=hof,verbose=True)
        return pop, log, hof

In [11]:
if __name__=='__main__':
    pop,log,hof = main()
    expr = tools.selBest(pop,1)[0]
    print(expr)

   	      	           fitness            	             size             
   	      	------------------------------	------------------------------
gen	nevals	avg 	max	min	std   	avg 	max	min	std   
0  	100   	8.93	33 	7  	3.1153	3.56	7  	2  	1.5318
1  	49    	9.41	33 	7  	3.67449	3.66	8  	2  	1.56984
2  	57    	10.45	38 	7  	5.78684	3.74	8  	1  	1.63475
3  	65    	12   	44 	7  	7.45654	4.01	10 	1  	1.95701
4  	59    	16.33	104	7  	13.4403	4.84	9  	1  	2.01851
5  	58    	21.07	104	7  	18.6704	5.33	10 	1  	2.20479
6  	62    	30.2 	199	7  	30.861 	5.94	11 	1  	2.39925
7  	53    	39.69	199	7  	44.9165	6.04	13 	1  	2.28   
8  	48    	51.32	199	7  	53.5548	6.18	15 	1  	2.13251
9  	43    	75   	199	7  	72.3003	5.89	11 	1  	2.05375
10 	62    	68.99	199	7  	76.5086	6.04	14 	3  	2.06359
11 	43    	90.41	199	7  	80.3006	6.49	15 	3  	2.28252
12 	53    	89.03	199	7  	82.7008	6.75	23 	3  	2.98454
13 	56    	93.84	199	7  	85.1171	6.78	23 	1  	3.10348
14 	57    	91.39	199	7  	87.923 	6.69	19 	1  	3.161

UnboundLocalError: local variable 'env' referenced before assignment